In [1]:
# Aaron
import numpy.random as nr
import itertools
from collections import namedtuple
import pandas as pd

In [2]:
start_time = 9.0 # 9am
end_time = 16.0 # 4pm
hourly_rate = 10/60 # every 10 minutes
process_time_min_max = (5/60, 20/60) # uniform between 10 and 30 minutes

In [3]:
def arrival_times():
    current_time = start_time
    while True:
        current_time += nr.exponential(hourly_rate)
        if current_time > end_time:
            break
        else:
            yield current_time
        
def process_times():
    while True: yield nr.uniform(*process_time_min_max)
        
def patients():
    return zip(arrival_times(), process_times())
        

In [4]:
for a in itertools.islice(patients(), 6):
    print(a)

(9.22647697452822, 0.2655797646475507)
(9.269022429514955, 0.18736198398352416)
(9.57737637227136, 0.18725573329773443)
(9.719719861923531, 0.2801266643451028)
(9.782588453675242, 0.08807369500432473)
(9.80840073728496, 0.3036979691681516)


In [5]:
Patient_record = namedtuple('Patient_record', 'arrival wait process_time')
def simulate():
    doctors = [start_time for _ in range(3)]
    for arrival_time, process_time in patients():
        # A patient has just arrived
        # Which doctor will see this patient?
        doctors = sorted(doctors)
        first_free_doctor = doctors.pop(0)
        # when will this patient be seen?
        enter_the_room = max(arrival_time, first_free_doctor)
        leave_the_room = enter_the_room + process_time
        # this doctor will be busy until then
        doctors.append(leave_the_room)
        assert len(doctors) == 3
        yield Patient_record(arrival_time, enter_the_room-arrival_time, process_time)

In [6]:
simulated_data = pd.DataFrame(list(simulate()))
simulated_data

,arrival,wait,process_time
0,9.057364,0.000000,0.314053
1,9.133635,0.000000,0.144422
2,9.245335,0.000000,0.282666
3,9.615008,0.000000,0.153809
4,9.835923,0.000000,0.206641
5,9.995190,0.000000,0.318543
6,10.202111,0.000000,0.106085
7,10.374703,0.000000,0.210055
8,10.428493,0.000000,0.324764
9,10.582662,0.000000,0.283858


In [7]:
Record_of_one_simulation = namedtuple('Record_of_one_simulation', 'n number_of_waiters average_wait_of_waiters closing_time')
def one_simulation():
    simulated_data = pd.DataFrame(list(simulate()))
    # How many patients came to the office?
    n = simulated_data.shape[0]
    # How many had to wait for a doctor?
    waiters = simulated_data.query('wait > 0')
    number_of_waiters = waiters.shape[0]
    # What was their average wait?
    average_wait_of_waiters = waiters.wait.mean()
    # When did the office close?
    closing_time = float(simulated_data.iloc[[-1],:].eval('arrival+wait+process_time'))
    rec = Record_of_one_simulation(n, number_of_waiters, average_wait_of_waiters, closing_time)
    return rec

In [8]:
number_of_simulations = 10
sims = pd.DataFrame([one_simulation() for sim in range(number_of_simulations)])

print(sims)

    n  number_of_waiters  average_wait_of_waiters  closing_time
0  50                 11                 0.074844     16.068317
1  45                  8                 0.077117     16.015736
2  39                  7                 0.079502     15.944850
3  45                  2                 0.083258     16.120998
4  44                  7                 0.077304     16.067006
5  42                  5                 0.040500     15.947436
6  37                  7                 0.076368     16.000642
7  36                  5                 0.114543     16.273268
8  51                  9                 0.056909     16.060058
9  40                  7                 0.104967     16.289932
